In [3]:
pip install arxiv

Note: you may need to restart the kernel to use updated packages.


create a wrapper based on wikipidea and arxiv


In [4]:
pip install wikipedia

In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [6]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)#top_k_result define how many result you want and doc _content_chars_max define how many character you want to show in the summary
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [7]:
wiki.name

'wikipedia'

using webbaseloader to load any data from any website 

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

loader=WebBaseLoader("https://docs.smith.langchain.com/")# the loader basically read the entire webpage and return it as a string
docs=loader.load() #load the entire content from the particular website
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs) #chunk_overlap=determines how much overlap there should be between consecutive chunks

embedding = OllamaEmbeddings(model="tinyllama") 
vectordb=FAISS.from_documents(documents,embedding)
#vectordb convert into retriever 
retriever=vectordb.as_retriever()

retriever
#retriever for wikipedia

USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\Biswajit\AppData\Local\Temp\ipykernel_16904\3099206296.py:10: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(model="tinyllama")


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B07F590220>, search_kwargs={})

In [9]:
#create a retriever tool
#Create a tool to do retrieval of documents.
from langchain.tools.retriever import create_retriever_tool
#it is basically creating a tool to do the search for that particular page 
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "search  for information about Langsmith.For any question about Langsmith you must use this tools")#retriever=retriever name,langsmith_search help to identify the tool name 

In [10]:
retriever_tool.name

'langsmith_search'

3rd Retriever tool:ARXIV platform 

In [11]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper


arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)#top_k_result define how many result you want and doc _content_chars_max define how many character you want to show in the summary
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)# create a arxiv wrrapper

In [12]:
arxiv.name

'arxiv'

Combining all the tools 

In [13]:
tools=[wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Biswajit\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='search  for information about Langsmith.For any question about Langsmith you must use this tools', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001B07F58E170>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vect

Step 2 :Query from the specific tools

OPENAI agent 

In [14]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0)

There are two way to creating a prompt one is chat prompt  template and one is langchain hub- this is a generic prompt which is crate by many other user of langchain and also crate by langchain team.

In [15]:
from langchain import hub
#get the prompt to use -you can modify this to suit your needs
prompt= hub.pull("hwchase17/openai-functions-agent")# 1st one is username that is prsent in the langchainhub  and other one is name of the function that you want to use
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

OPEN_AI TOOL AGENT

In [16]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [17]:
#to use this agent we have to use agent executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)#agentname ,tools name ,verbose help to get the details whenever I am getting a respone
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [25]:
#in order to execute anything
from openai import RateLimitError
import requests

try:
    # Attempt to execute agent
    result = agent_executor.invoke({"input": "Tell me about langsmith"}) #so this agent will excute and interact from the tool that is related to langsmith (Webbaseloader)
    print(result)

except requests.exceptions.RequestException as e: # his is the first except block. It catches exceptions that occur during the HTTP request itself (e.g., network errors, connection errors, etc.).
    print(f"Error: {e}") # Return the error message.

except (KeyError, ValueError) as e: #:: This is the second except block. It catches KeyError(e.g., if the API response doesn't have the "output" or "content" keys) and ValueErro(e.g., if the API returns an invalid JSON string) exceptionss break down the get_openai_response and `get_ollama.
    print(f"Error parsing response: {e}")

except RateLimitError as e:
    # Handle quota exceeded specifically
    print("Naukri karla fir bharna payment")

except Exception as e:
    # Optional: handle other unexpected errors gracefully
    print(f"An unexpected error occurred: {e}")






> Entering new AgentExecutor chain...
Naukri karla fir bharna payment
